# Beer Classification

For this project we will cluster the user from whom we will take review for three beer and use that information to suggest him beer.

In [1]:
#For this project we will only use panda and numpy
import pandas as pd
import numpy as np

In [2]:
# For further reference we will be saving a copy in our system
df = pd.read_csv("out.CSV", encoding='latin-1')

In [3]:
from datetime import datetime
startTime = datetime.now()

In [4]:
#since the data is too large we will trime the data and use only review_overall, review_profilename and beer_name
cleandata = df.loc[:,["rating","profilename", "beer"]]
cleandata.head()

,rating,profilename,beer
0,1.5,stcules,Sausa Weizen
1,3.0,stcules,Red Moon
2,3.0,stcules,Black Horse Black Beer
3,3.0,stcules,Sausa Pils
4,4.0,johnmichaelsen,Cauldron DIPA


In [5]:
# In future this value will be updated from the web interace or the website.
#For testing purpose we will give our own value
beer1 = "Budweiser"
overall1 = 1
beer2 = "Weihenstephaner Hefeweissbier"
overall2 = 5
beer3 = "Trappistes Rochefort 6"
overall3 = 3

In [6]:
#Te panda throw same wrong alerts which stops creating a new column in the codes. So we will put it as none
pd.options.mode.chained_assignment = None

## Reduce the size of the dataset

In [7]:
#Here we will try to reduce the dataset. Initally we will try find all the user who reviewed the beer 1
first = cleandata[(cleandata.beer == beer1)]


#Since we will be calculationg Eucledian distance we will find the difference of the review_Overall 
#and overall review given by the user
first['score'] = (first.rating - overall1)**2
first.head()


,rating,profilename,beer,score
1286802,3.0,Thehurt01,Budweiser,4.00
1301942,3.0,marc956,Budweiser,4.00
1305639,2.5,robertp1024,Budweiser,2.25
1305812,1.5,arguemaniac,Budweiser,0.25
1306083,3.0,ralphw56,Budweiser,4.00


In [8]:
#We will reduce the data set to find all the user who reviewed the beer 2
second = cleandata[(cleandata.beer == beer2)]

#Since we will be calculationg Eucledian distance we will find the difference of the review_Overall 
#and overall review given by the user
second['score'] = (second.rating - overall2)**2

second.head()

,rating,profilename,beer,score
669204,5.0,rtepiak,Weihenstephaner Hefeweissbier,0.00
679191,5.0,Casconi,Weihenstephaner Hefeweissbier,0.00
679392,4.5,drseamus,Weihenstephaner Hefeweissbier,0.25
682090,4.5,Overlord,Weihenstephaner Hefeweissbier,0.25
682190,5.0,Stephen63,Weihenstephaner Hefeweissbier,0.00


In [9]:
#We will reduce the data set to find all the user who reviewed the beer 
third = cleandata[(cleandata.beer == beer3)]

#Since we will be calculationg Eucledian distance we will find the difference of the review_Overall 
#and overall review given by the user
third['score'] = (third.rating - overall3)**2

third.head()

,rating,profilename,beer,score
339308,4.0,Wildman,Trappistes Rochefort 6,1.00
340750,4.0,sonicdescent,Trappistes Rochefort 6,1.00
341214,3.5,tweeder263,Trappistes Rochefort 6,0.25
341633,4.0,aracauna,Trappistes Rochefort 6,1.00
342237,4.0,frank4sail,Trappistes Rochefort 6,1.00


In [10]:
#We have list of of the user who reviewed all the three beer we will append to create one list
firstdata = first.append(second)
final = firstdata.append(third)
final.head()

,rating,profilename,beer,score
1286802,3.0,Thehurt01,Budweiser,4.00
1301942,3.0,marc956,Budweiser,4.00
1305639,2.5,robertp1024,Budweiser,2.25
1305812,1.5,arguemaniac,Budweiser,0.25
1306083,3.0,ralphw56,Budweiser,4.00


# Picking the top users

In [11]:
#We will use grouby function with count function to find the how many beer did the user reviewed
grouppeddata = (final.groupby( ["profilename"] ).count())

#Using below condition we will get all the user who have reiewed all the three beers
grp = grouppeddata[grouppeddata.rating == 3].reset_index()
grp.head()


,profilename,rating,beer,score
0,1fastz28,3,3,3
1,ADR,3,3,3
2,AKBelgianBeast,3,3,3
3,BMoney575,3,3,3
4,BedetheVenerable,3,3,3


In [12]:
#We will now get all the name of the reiwer and convert it into a list for further processing
reviewer = grp.loc[:,"profilename"]
reviewer = reviewer.values.tolist()

In [13]:
#We will create a for loop to get all the detail of user who reviwed all the three beer
finalresut = pd.DataFrame()
for allthename in reviewer:
    beername = allthename
    thefinalname = final[(final.profilename == beername)]
    finalresut = finalresut.append(thefinalname)
finalresut.shape

(561, 4)

## Finding the distance

In [14]:
#We see now our result is reduced with all those user who have reviwed all the three beer with score which is square of 
#differnce between review_overall and the review given by the user
grouppeddata1 = finalresut.groupby( [ "profilename"] ).sum() 
#We will not use square root on this as we are using the score less than 1
#We know that square root of zero and 1 is same, so we will skip that step
grouppeddata1 = grouppeddata1[grouppeddata1.score < 1].reset_index()
#We now have the cluster or name of user whosed eucledian distance is less than 1 and similar to our user who reviwed
grouppeddata1

,profilename,rating,score
0,Bierman9,9.0,0.50
1,MrHungryMonkey,10.0,0.50
2,harrymel,9.0,0.50
3,kkipple,15.0,0.00
4,mynie,7.5,0.25
5,wnhay,11.0,0.00


In [15]:
#We will again extract the name of of the reviewer and create alist
reviewer = grouppeddata1.loc[:,"profilename"]
reviewer = reviewer.values.tolist()
reviewer

['Bierman9', 'MrHungryMonkey', 'harrymel', 'kkipple', 'mynie', 'wnhay']

In [16]:
#With the list we now select all the beer these reviewer have reviewed
finalresut = pd.DataFrame()
for allthename in reviewer:
    beername = allthename
    thefinalname = cleandata[(cleandata.profilename == beername)]
    finalresut = finalresut.append(thefinalname)
finalresut.shape

(5563, 3)

## Letting the beer Vote

In [17]:
#Now we will use group function, So each beer can vote to sum up the value of overall rating
grouppeddata2 = cleandata.groupby( [ "beer"] ).sum() 
#We will use cindition of all the beer whose total rating is more than 100 to shorten the list we are working on
grouppeddata2 = grouppeddata2[grouppeddata2.rating > 100].reset_index()

## Picking the Top 10 beer

In [18]:
#using sort function we will sort the values such that all the beer with highest review_overall is in top
theresultofgod = grouppeddata2.sort_values(by = "rating", ascending=False)
#Now we use the head function and loc to get top 10 beer for the user
toprint = theresultofgod.head(10)
reviewer = toprint.loc[:,"beer"]
reviewer = reviewer.values.tolist()
reviewer

['90 Minute IPA',
 'Old Rasputin Russian Imperial Stout',
 'Sierra Nevada Celebration Ale',
 'India Pale Ale',
 'Two Hearted Ale',
 'Pliny The Elder',
 'Stone Ruination IPA',
 'Arrogant Bastard Ale',
 'Sierra Nevada Pale Ale',
 'Stone IPA (India Pale Ale)']

In [19]:
print(datetime.now() - startTime)

0:00:01.940713
